# Time Series window data 생성 및 모델별 성능 비교

1. Persistence algorithm 을 이용한 forecasting 
2. simple 1 layer NN 을 이용한 forecasting  
3. Deep NN을  이용한 forecasting  
4. LSTM 을  이용한 forecasting

## Sunspots 데이터셋을 활용하여 window_dataset 만들기

- 시간 절약을 위해 최근 500 개월치 data 만 이용

time 변수는 DataFrame의 마지막 500개 행의 첫 번째 열(0번 인덱스)에 해당하는 값을 포함하고 있습니다. time_series 변수는 마지막 500개 행의 두 번째 열(1번 인덱스)에 해당하는 값을 포함하고 있습니다.

## Supervised Learning dataset 작성

- Persistent Algorithm 과 비교하기 위해 Many-to-One style 로 예측 

windowed_dataset 함수는 다음 단계를 수행합니다:

1. series를 입력으로 받아 tf.data.Dataset.from_tensor_slices() 함수를 사용하여 TensorFlow Dataset 객체를 생성합니다.  
2. ds.window() 함수를 사용하여 데이터를 겹치는 윈도우로 분할합니다. 윈도우 크기는 window_size+1로 설정하며, 각 윈도우는 1씩 이동(shift)합니다. drop_remainder=True를 설정하여 길이가 부족한 마지막 윈도우를 제거합니다.  
3. ds.flat_map() 함수를 사용하여 각 윈도우를 일련의 데이터 포인트로 평평하게(flatten) 만듭니다.  
4. ds.shuffle() 함수를 사용하여 데이터를 섞습니다. shuffle_buffer 매개 변수는 셔플링에 사용되는 버퍼 크기를 결정합니다.  
5. ds.map() 함수를 사용하여 각 윈도우의 마지막 데이터 포인트를 레이블로 설정하고 나머지를 입력 특성으로 사용합니다. 즉, 각 윈도우에서 하나의 출력 값(single step output)이 생성됩니다.  
6. ds.batch() 함수를 사용하여 데이터를 batch_size 크기의 배치로 구성하고, ds.prefetch(1)을 사용하여 학습 중에 다음 배치를 미리 가져옵니다.  

### train/validation set 분리

## 1. Persistence algorithm 을 이용한 forecasting 

- Persistence Algorithm : last value 를 next value 로 예측에 사용  

naive_forecast는 주어진 시계열 데이터의 단순한 예측값을 생성합니다. 여기서는 테스트 데이터셋의 이전 값(즉, 마지막 훈련 데이터셋의 값)을 사용하여 테스트 데이터셋을 예측합니다. 이 예측 방법은 매우 단순하며 실제로는 복잡한 패턴을 예측할 수 없지만, 복잡한 모델과의 성능 비교를 위한 기준점으로 사용할 수 있습니다.

naive_forecast에 저장된 처음 10개의 예측값은 다음과 같습니다:

테스트 데이터셋과 naive_forecast 사이의 평균 제곱 오차(Mean Squared Error, MSE)와 평균 절대 오차(Mean Absolute Error, MAE)를 계산합니다.  
일반적으로 낮은 값이 더 좋은 예측 성능을 나타냅니다.

## 2. simple 1 layer NN 을 이용한 forecasting - Linear Regression

input_shape: 입력 데이터의 형태를 지정합니다. 여기서 입력 데이터의 형태는 [window_size]입니다.
이 모델은 각 시계열 윈도우에 대해 하나의 출력 값을 예측합니다. 이 간단한 모델은 선형 회귀와 유사하게 작동하며, 시계열 데이터의 간단한 패턴을 포착할 수 있습니다. 

시계열 데이터를 다룰 때, 우리는 전체 데이터를 훈련 데이터와 테스트 데이터로 나눕니다. 이 과정에서 'window_size'라는 개념을 사용하는데, 이는 한 번에 고려할 데이터의 범위(길이)를 의미합니다. 'split_time'은 훈련 데이터와 테스트 데이터를 나누는 지점을 나타냅니다.

예를 들어, 'split_time'이 450이고 'window_size'가 30일 때, 'split_time - window_size' 계산은 420이 됩니다. 이는 훈련 데이터에서 마지막으로 고려할 데이터 범위가 시계열 데이터의 420번째 위치에서 시작한다는 의미입니다. 즉, 이 위치부터 'window_size'만큼의 데이터를 마지막 훈련 데이터로 사용합니다.

시계열 데이터의 전체 길이는 'len(time_series)'로 확인할 수 있습니다. 이 정보를 바탕으로, 테스트 데이터를 예측할 때는 'X_test'의 길이만큼 예측을 진행합니다. 그리고 예측 결과의 마지막 부분을 'time_series[-len(X_test):]'를 사용하여 추출합니다. 이렇게 하면 예측 결과와 'X_test'의 길이가 같아져서, 실제 값과 예측 값을 비교하기 쉬워집니다.

간단히 말해서, 우리는 시계열 데이터의 끝부분에서 'window_size'만큼 떨어진 위치를 시작점으로 삼아 테스트 데이터를 예측하고, 이 예측 결과 중 마지막 부분을 사용하여 실제 테스트 데이터와 비교합니다.

이 코드는 시계열 데이터에서 테스트 데이터에 해당하는 부분의 예측값을 계산합니다. 예측 과정은 `split_time` 지점부터 시계열 데이터의 끝까지 진행되며, 각 예측에는 바로 앞의 `window_size` 만큼의 데이터가 사용됩니다.

예를 들어, 만약 `split_time`이 450이고 시계열 데이터의 총 길이(`len(time_series)`)가 500이라면, 이 코드는 450번째 시점부터 499번째 시점까지, 즉 테스트 데이터에 해당하는 각 시점에 대해 예측을 수행합니다. 결과적으로 `forecast` 리스트에는 총 50개의 예측값이 저장되며, 이는 테스트 데이터의 길이인 500에서 450을 뺀 값과 일치합니다.

## 3. Deep NN을  이용한 forecasting

## 4. LSTM 을  이용한 forecasting

- RNN 은 3 Dimension input 이 필요하므로 Lambda layer 를 이용하여 2D $\rightarrow$ 3D expand  

### 4-1. Simple LSTM

- 첫 번째 람다 레이어는 차원을 확장하는 데 도움이 됩니다. 여기서는 배열을 1차원 확장하고 모델이 모든 길이의 시퀀스를 사용할 수 있음을 의미하는 input_shape=[None]을 설정 합니다.

- 최종 람다 계층은 출력을 100배 확장하여 훈련에 도움이 될 수 있습니다. RNN의 기본 레이어는 -1과 1 사이의 값을 출력하는 tanh입니다. 출력을 스케일링하면 이러한 값을 일반적으로 40, 50, 60 등의 값인 시계열에 더 가깝게 만들 수 있습니다.

### 4-2. Stacked + Bidirectoinal LSTM

이 코드는 양방향 LSTM 모델을 정의합니다. 여기에 포함된 각 층은 다음과 같습니다:

Lambda 층: 입력 데이터의 마지막 축을 확장하여 형태를 [None]에서 [None, 1]로 변경합니다. 이렇게 하면 입력 데이터를 2D에서 3D로 변환하여 LSTM 층에 적합한 형태로 만듭니다.

Bidirectional 층: 양방향 LSTM을 사용하여 시계열 데이터를 처리합니다. 첫 번째 양방향 LSTM 층은 32개의 유닛을 가지며, return_sequences=True로 설정되어 있어 시퀀스 전체를 출력합니다. 이렇게 하면 이 층의 출력이 다음 LSTM 층의 입력으로 사용될 수 있습니다.

또 다른 Bidirectional 층: 두 번째 양방향 LSTM 층 역시 32개의 유닛을 가지고 있지만, return_sequences의 기본값인 False로 설정되어 마지막 시간 단계의 출력만 반환합니다.

Dense 층: 출력 유닛이 1개인 완전 연결 층입니다. 이 층은 LSTM 층에서 얻은 정보를 바탕으로 시계열 데이터의 다음 값을 예측하는 역할을 합니다.

마지막 Lambda 층: 출력 값을 100배로 스케일업합니다. 이렇게 하면 네트워크가 더 큰 값의 범위를 예측할 수 있게 됩니다.